Examen Bimestral

In [ ]:
# Read data
import pandas as pd

In [6]:
# Leer el archivo JSON
url = '../data/arxiv_examen.json'
data = pd.read_json(url, encoding='utf-8', lines=True)

In [7]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Descargar recursos necesarios de nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/murder/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/murder/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/murder/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [8]:
print(data)

             id                                              title  \
0      704.0001  Calculation of prompt diphoton production cros...   
1      704.0002           Sparsity-certifying Graph Decompositions   
2      704.0003  The evolution of the Earth-Moon system based o...   
3      704.0004  A determinant of Stirling cycle numbers counts...   
4      704.0005  From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...   
...         ...                                                ...   
16995  707.3825  Emergence of noncollinear magnetic ordering in...   
16996  707.3826                      More hilltop inflation models   
16997  707.3827  Engineering Silicon Nanocrystals: Theoretical ...   
16998  707.3828  Structure, bonding and magnetism in cobalt clu...   
16999  707.3829  Occupation Statistics of Critical Branching Ra...   

                                                abstract  
0        A fully differential calculation in perturba...  
1        We describe a new algorithm, the

In [9]:
# Definir stopwords en inglés
stop_words = set(stopwords.words('english'))

# Función para preprocesar texto
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Tokenizar palabras
    tokens = word_tokenize(text)
    # Eliminar stopwords y signos de puntuación
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    return tokens

# Aplicar función al título y resumen
data['title_tokens'] = data['title'].apply(preprocess_text)
data['abstract_tokens'] = data['abstract'].apply(preprocess_text)

# Mostrar resultado
print(data[['title_tokens', 'abstract_tokens']])

                                            title_tokens  \
0      [calculation, prompt, diphoton, production, cr...   
1           [sparsity-certifying, graph, decompositions]   
2      [evolution, earth-moon, system, based, dark, m...   
3      [determinant, stirling, cycle, numbers, counts...   
4           [dyadic, \lambda_, \alpha, \lambda_, \alpha]   
...                                                  ...   
16995  [emergence, noncollinear, magnetic, ordering, ...   
16996                       [hilltop, inflation, models]   
16997  [engineering, silicon, nanocrystals, theoretic...   
16998  [structure, bonding, magnetism, cobalt, clusters]   
16999  [occupation, statistics, critical, branching, ...   

                                         abstract_tokens  
0      [fully, differential, calculation, perturbativ...  
1      [describe, new, algorithm, k, \ell, -pebble, g...  
2      [evolution, earth-moon, system, described, dar...  
3      [show, determinant, stirling, cycle,

In [30]:
# Vectorizar
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
# Fit on the combined text from title_tokens and abstract_tokens
vectorizer.fit(data['title_tokens'].astype(str) + ' ' + data['abstract_tokens'].astype(str))
corpus_vect = vectorizer.transform(data['title_tokens'].astype(str) + ' ' + data['abstract_tokens'].astype(str)) # Transformación
print(corpus_vect.shape)

(17000, 44149)


In [11]:
from sklearn.metrics.pairwise import cosine_similarity

In [45]:
# Función para buscar usando TF-IDF
def search_tfidf(query, top_k=10):
    # Vectorize the query
    query_vect = vectorizer.transform([query])

    # Calculate cosine similarities between the query and documents
    cosine_similarities = cosine_similarity(query_vect, corpus_vect).flatten()

    # Create a DataFrame with the results
    df_results = pd.DataFrame({'Identificador': data['id'], 'Título': data['title'], 'Resumen': data['abstract'], 'Similitud coseno': cosine_similarities})

    # Sort by similarity and get the top results
    df_results = df_results.sort_values(by='Similitud coseno', ascending=False)
    return df_results.head(top_k)

In [46]:
query = "diphoton production cross sections"

In [47]:
search_tfidf(query)

,Identificador,Título,Resumen,Similitud coseno
0,704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,0.393993
15464,707.2294,Search for a High-Mass Diphoton State and Limi...,We have performed a search for new particles...,0.362845
9537,706.0851,Electroweak measurements at the Tevatron,The increasing size of the data samples reco...,0.332994
8315,705.4313,Projectile Fragmentation of $^{86}$Kr at 64 Me...,We measured fragmentation cross sections pro...,0.306359
11499,706.2813,Measurement of the Total Hadronic Cross Sectio...,"Using the CLEO III detector, we measure abso...",0.285362
10803,706.2117,Novel Master Formula for Twist-3 Soft-Gluon-Po...,We prove that twist-3 soft-gluon-pole (SGP) ...,0.264567
4351,705.0349,New isotope 44Si and systematics of the produc...,The results of measurements of the productio...,0.257428
11379,706.2693,Reaction cross sections for proton scattering ...,Microscopic optical model potential results ...,0.252194
8251,705.4249,Extrapolation of neutron-rich isotope cross-se...,Using the measured fragmentation cross secti...,0.251028
5053,705.1051,Quasi-elastic neutrino charged-current scatter...,The charged-current quasi-elastic scattering...,0.245088


In [48]:
# BM25 Search
from rank_bm25 import BM25Okapi

def search_bm25(query,  top_k=10):
  # Tokenize the corpus for BM25
  tokenized_corpus = data['abstract_tokens'].tolist()

  bm25_doc = BM25Okapi(tokenized_corpus)
  scores = bm25_doc.get_scores(query)

  # Create a DataFrame with the data
  df = pd.DataFrame({'Identificador': data['id'], 'Título': data['title'], 'Resumen': data['abstract'], 'Score BM25': scores})
  df = df.sort_values(by='Score BM25', ascending=False)
  bm25_results = df.head(top_k)
  bm25_results
  return bm25_results

In [49]:
search_bm25(query)

,Identificador,Título,Resumen,Score BM25
14426,707.1256,The geometry of the critical set of nonlinear ...,We study the critical set C of the nonlinear...,54.005663
13534,707.0364,Polarization types of isogenous Prym-Tyurin va...,"Let p:C-->Y be a covering of smooth, project...",49.654692
14278,707.1108,Permutation binomials over finite fields,We prove that if x^m + c*x^n permutes the pr...,46.431785
4327,705.0325,The order of the largest complete minor in a r...,Let ccl(G) denote the order of the largest c...,43.879518
12788,706.4102,Ramsey numbers and the size of graphs,"For two graph H and G, the Ramsey number r(H...",43.550299
16486,707.3316,Morita equivalences of cyclotomic Hecke algebr...,We prove a Morita reduction theorem for the ...,43.228933
7751,705.3749,Difference sets and shifted primes,"We show that if A is a subset of {1, ..., n}...",42.580936
14272,707.1102,Nonexistence of permutation binomials of certa...,Suppose x^m + c*x^n is a permutation polynom...,41.792553
15782,707.2612,Exceptional covers of surfaces,Consider a finite morphism f:X -> Y of smoot...,39.846724
16842,707.3672,Products of irreducible random matrices in the...,We consider the recursive equation ``x(n+1)=...,39.735954


In [18]:
# Faiss
from sentence_transformers import SentenceTransformer
import faiss
import os
from dotenv import load_dotenv
from openai import OpenAI

In [19]:
# Create embeddings for the recipes data
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Generando embeddings...")
embeddings = model.encode(data['abstract'].tolist(), convert_to_numpy=True)

Generando embeddings...


In [20]:
# Create index FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance
index.add(embeddings)  # Add embeddings to the index

In [21]:
# Config OpenAI
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [50]:
# Search function
def search_faiss(query, top_k=10):
    # Create embedding for the query
    query_embedding = model.encode([query], convert_to_numpy=True)

    # Search in the index
    distances, indices = index.search(query_embedding, top_k)

    # Get the results from the original data
    results = data.iloc[indices[0]].copy()
    results['Distance'] = distances[0]
    
    return results[['id', 'title', 'abstract', 'Distance']]

In [51]:
# Search function
search_faiss(query)

,id,title,abstract,Distance
15545,707.2375,Pion Production by Protons on a Thin Beryllium...,An analysis of inclusive pion production in ...,1.046610
6746,705.2744,Distributions for MSSM Higgs boson + jet produ...,We present pseudorapidity and transverse mom...,1.051520
9537,706.0851,Electroweak measurements at the Tevatron,The increasing size of the data samples reco...,1.055434
7886,705.3884,Inclusive electron spectrum in the region of p...,We have carried out a calculation of the inc...,1.059579
1984,704.1985,Electromagnetic Higgs production,The cross section for central diffractive Hi...,1.066204
13288,707.0118,Proton Structure Functions at High $Q^{2}$ and...,Neutral and charged current deep inelastic s...,1.066468
14716,707.1546,"Producing an Intense, Cool Muon Beam via e+e- ...",We consider a highly unconventional approach...,1.097358
9387,706.0701,Top Pair Production cross-section at the Tevatron,An overview of latest top quark pair product...,1.117400
6433,705.2431,Higher-order Threshold Corrections for Single ...,I discuss single top quark production at the...,1.122415
1428,704.1429,Light stops in the MSSM parameter space,We consider the regions of the MSSM paramete...,1.133265


In [24]:
# RAG
def rag_search(query, top_k=3):
    # Get documents
    docs = search_faiss(query, top_k=top_k)
    # Create a prompt for OpenAI
    prompt = f"""
    Basándote ÚNICAMENTE en los siguientes documentos relevantes, responde la consulta del usuario.

    CONSULTA: "{query}"

    DOCUMENTOS RELEVANTES:
    """
    
    for idx, (_, row) in enumerate(docs.iterrows(), 1):
        prompt += f"""
    Documento {idx}:
    Título: {row['title']}
    Resumen: {row['abstract']}
    Distancia semántica: {row['Distance']:.4f}
    ---
    """
    
    prompt += """
    INSTRUCCIONES:
    1. Resume la información más relevante de estos documentos para responder la consulta
    2. Explica por qué estos documentos son relevantes para la consulta
    3. Identifica los conceptos clave y metodologías mencionadas
    4. Si hay resultados o conclusiones importantes, inclúyelos
    5. Mantén un enfoque científico y preciso
    
    Estructura tu respuesta en:
    - Resumen de hallazgos principales
    - Relevancia de los documentos
    - Conceptos clave identificados
    """
    # Generate response using OpenAI
    response = client.responses.create(
        model="gpt-4.1",
        input=prompt
    )
    return response.output_text

In [27]:
response = rag_search(query)
print("Respuesta de ChatGPT:")
print(response)

Respuesta de ChatGPT:
**Resumen de hallazgos principales:**

Ninguno de los documentos relevantes proporciona una medición directa o cálculo explícito de las secciones eficaces (cross sections) de producción de "diphotons" (pares de fotones). Sin embargo, los documentos abordan temas relacionados, como la medición de secciones eficaces diferenciales en distintos procesos en colisionadores, la producción de bosones y dibosones (pares de bosones, entre ellos posibles combinaciones de bosones electrodébiles) y la producción de partículas junto a jet en escenarios del Modelo Estándar y MSSM.

- El Documento 1 estudia la producción de piones en colisiones protón-berilio a diversas energías y proporciona mediciones detalladas de las secciones eficaces diferenciales para estos procesos.
- El Documento 2 aborda la producción del bosón de Higgs neutral más ligero del MSSM asociado con un jet y analiza las distribuciones de secciones eficaces diferenciales (en pseudorapidez y momento transversal

In [65]:
def compare_search_methods(query, top_k=10):
    print(f"=== COMPARACIÓN DE MÉTODOS PARA: '{query}' ===\n")
    
    # 1. Obtener resultados de cada método
    tfidf_results = search_tfidf(query)
    tfidf_ids = tfidf_results['Identificador'].tolist()
        
    bm25_results = search_bm25(query)
    bm25_ids = bm25_results['Identificador'].tolist()
    
    faiss_results = search_faiss(query, top_k)
    faiss_ids = faiss_results['id'].tolist()

    # 2. Análisis de documentos en común (por ID)
    print("\n📊 ANÁLISIS DE DOCUMENTOS EN COMÚN (por ID):")
    print("=" * 60)
    
    # Intersecciones entre métodos usando IDs
    tfidf_bm25_common = set(tfidf_ids) & set(bm25_ids)
    tfidf_faiss_common = set(tfidf_ids) & set(faiss_ids)
    bm25_faiss_common = set(bm25_ids) & set(faiss_ids)
    all_common = set(tfidf_ids) & set(bm25_ids) & set(faiss_ids)

    print(f"• TF-IDF ∩ BM25: {len(tfidf_bm25_common)} documentos")
    print(f"• TF-IDF ∩ FAISS: {len(tfidf_faiss_common)} documentos")
    print(f"• BM25 ∩ FAISS: {len(bm25_faiss_common)} documentos")
    print(f"• Común a los 3 métodos: {len(all_common)} documentos")
    
    # 3. Mostrar documentos comunes a los 3 métodos
    if all_common:
        print(f"\n🎯 DOCUMENTOS COMUNES A LOS 3 MÉTODOS:")
        print("-" * 80)
        for i, doc_id in enumerate(all_common, 1):
            # Buscar el documento por ID
            doc_info = data[data['id'] == doc_id].iloc[0]
            print(f"{i}. ID: {doc_id}")
            print(f"   TÍTULO: {doc_info['title']}")
            print(f"   RESUMEN: {doc_info['abstract'][:150]}...")
            print(f"   {'='*70}")
    else:
        print("❌ No hay documentos comunes a los 3 métodos")
    
    # 4. Mostrar documentos comunes por pares
    print(f"\n📋 DOCUMENTOS COMUNES POR PARES:")
    print("-" * 60)
    
    # Solo en TF-IDF y BM25 (no en FAISS)
    only_tfidf_bm25 = tfidf_bm25_common - all_common
    if only_tfidf_bm25:
        print(f"\n🔸 Solo en TF-IDF y BM25 ({len(only_tfidf_bm25)} docs):")
        for doc_id in list(only_tfidf_bm25)[:3]:  # Mostrar solo los primeros 3
            doc_title = data[data['id'] == doc_id]['title'].iloc[0]
            print(f"   • ID {doc_id}: {doc_title[:60]}...")
    
    # Solo en TF-IDF y FAISS (no en BM25)
    only_tfidf_faiss = tfidf_faiss_common - all_common
    if only_tfidf_faiss:
        print(f"\n🔸 Solo en TF-IDF y FAISS ({len(only_tfidf_faiss)} docs):")
        for doc_id in list(only_tfidf_faiss)[:3]:
            doc_title = data[data['id'] == doc_id]['title'].iloc[0]
            print(f"   • ID {doc_id}: {doc_title[:60]}...")
    
    # Solo en BM25 y FAISS (no en TF-IDF)
    only_bm25_faiss = bm25_faiss_common - all_common
    if only_bm25_faiss:
        print(f"\n🔸 Solo en BM25 y FAISS ({len(only_bm25_faiss)} docs):")
        for doc_id in list(only_bm25_faiss)[:3]:
            doc_title = data[data['id'] == doc_id]['title'].iloc[0]
            print(f"   • ID {doc_id}: {doc_title[:60]}...")

    # 5. Análisis de ordenamiento por ID
    print(f"\n📈 COMPARACIÓN DE ORDENAMIENTO POR ID:")
    print("=" * 80)
    
    # Crear tabla de comparación con IDs y títulos
    comparison_data = []
    for pos in range(0, top_k):
        row = {'Posición': pos + 1}
        
        # TF-IDF
        if pos < len(tfidf_ids):
            tfidf_id = tfidf_ids[pos]
            tfidf_title = data[data['id'] == tfidf_id]['title'].iloc[0]
            row['TF-IDF'] = f"ID {tfidf_id}: {tfidf_title[:35]}..."
        else:
            row['TF-IDF'] = "N/A"
            
        # BM25
        if pos < len(bm25_ids):
            bm25_id = bm25_ids[pos]
            bm25_title = data[data['id'] == bm25_id]['title'].iloc[0]
            row['BM25'] = f"ID {bm25_id}: {bm25_title[:35]}..."
        else:
            row['BM25'] = "N/A"
            
        # FAISS
        if pos < len(faiss_ids):
            faiss_id = faiss_ids[pos]
            faiss_title = data[data['id'] == faiss_id]['title'].iloc[0]
            row['FAISS'] = f"ID {faiss_id}: {faiss_title[:35]}..."
        else:
            row['FAISS'] = "N/A"
            
        comparison_data.append(row)
    
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))
    
    # 6. Métricas de similitud usando IDs
    print(f"\n📊 MÉTRICAS DE SIMILITUD POR ID:")
    print("-" * 50)
    
    def similarity_ids(list1, list2, k=top_k):
        set1 = set(list1[:k])
        set2 = set(list2[:k])
        intersection = len(set1 & set2)
        union = len(set1 | set2)
        return intersection / union if union > 0 else 0
    
    tfidf_bm25_sim = similarity_ids(tfidf_ids, bm25_ids)
    tfidf_faiss_sim = similarity_ids(tfidf_ids, faiss_ids)
    bm25_faiss_sim = similarity_ids(bm25_ids, faiss_ids)
    
    print(f"• Similitud TF-IDF vs BM25: {tfidf_bm25_sim:.3f}")
    print(f"• Similitud TF-IDF vs FAISS: {tfidf_faiss_sim:.3f}")
    print(f"• Similitud BM25 vs FAISS: {bm25_faiss_sim:.3f}")
    
    # 7. Resumen de diferencias
    print(f"\n📝 RESUMEN DE DIFERENCIAS:")
    print("=" * 50)
    print("• TF-IDF: Enfoque estadístico, mejor para coincidencias exactas de términos")
    print("• BM25: Mejora de TF-IDF, considera frecuencia de documentos y longitud")
    print("• FAISS: Búsqueda semántica, encuentra documentos conceptualmente similares")
    
    # 8. Análisis de rendimiento
    print(f"\n⚡ ANÁLISIS DE RENDIMIENTO:")
    print("-" * 50)
    print(f"• Documentos únicos por método:")
    print(f"  - Solo TF-IDF: {len(set(tfidf_ids) - set(bm25_ids) - set(faiss_ids))}")
    print(f"  - Solo BM25: {len(set(bm25_ids) - set(tfidf_ids) - set(faiss_ids))}")
    print(f"  - Solo FAISS: {len(set(faiss_ids) - set(tfidf_ids) - set(bm25_ids))}")


In [66]:
# Ejecutar comparación
comparison_results = compare_search_methods(query, top_k=10)

=== COMPARACIÓN DE MÉTODOS PARA: 'diphoton production cross sections' ===


📊 ANÁLISIS DE DOCUMENTOS EN COMÚN (por ID):
• TF-IDF ∩ BM25: 0 documentos
• TF-IDF ∩ FAISS: 1 documentos
• BM25 ∩ FAISS: 0 documentos
• Común a los 3 métodos: 0 documentos
❌ No hay documentos comunes a los 3 métodos

📋 DOCUMENTOS COMUNES POR PARES:
------------------------------------------------------------

🔸 Solo en TF-IDF y FAISS (1 docs):
   • ID 706.0851: Electroweak measurements at the Tevatron...

📈 COMPARACIÓN DE ORDENAMIENTO POR ID:
 Posición                                              TF-IDF                                                BM25                                               FAISS
        1 ID 704.0001: Calculation of prompt diphoton prod... ID 707.1256: The geometry of the critical set of... ID 707.2375: Pion Production by Protons on a Thi...
        2 ID 707.2294: Search for a High-Mass Diphoton Sta... ID 707.0364: Polarization types of isogenous Pry... ID 705.2744: Distributions for 

In [67]:
def measure_ranking_overlap(query, top_k=10):
    """
    Mide similitud entre rankings contando cuántos documentos del top-k coinciden
    """
    print(f"=== SIMILITUD ENTRE RANKINGS (Top-{top_k}) ===")
    print(f"Consulta: '{query}'\n")
    
    # 1. Obtener resultados de cada método
    tfidf_results = search_tfidf(query, top_k)
    tfidf_ids = set(tfidf_results['Identificador'].tolist())
    
    bm25_results = search_bm25(query, top_k)
    bm25_ids = set(bm25_results['Identificador'].tolist())
    
    faiss_results = search_faiss(query, top_k)
    faiss_ids = set(faiss_results['id'].tolist())
    
    # 2. Contar documentos coincidentes
    print("📊 DOCUMENTOS COINCIDENTES:")
    print("=" * 40)
    
    # Intersecciones entre pares
    tfidf_bm25_overlap = len(tfidf_ids & bm25_ids)
    tfidf_faiss_overlap = len(tfidf_ids & faiss_ids)
    bm25_faiss_overlap = len(bm25_ids & faiss_ids)
    
    # Intersección de los tres métodos
    all_three_overlap = len(tfidf_ids & bm25_ids & faiss_ids)
    
    print(f"TF-IDF ∩ BM25:     {tfidf_bm25_overlap}/{top_k} documentos ({tfidf_bm25_overlap/top_k*100:.1f}%)")
    print(f"TF-IDF ∩ FAISS:    {tfidf_faiss_overlap}/{top_k} documentos ({tfidf_faiss_overlap/top_k*100:.1f}%)")
    print(f"BM25 ∩ FAISS:      {bm25_faiss_overlap}/{top_k} documentos ({bm25_faiss_overlap/top_k*100:.1f}%)")
    print(f"Común a los 3:     {all_three_overlap}/{top_k} documentos ({all_three_overlap/top_k*100:.1f}%)")
    
    # 3. Calcular similitud normalizada (0-1)
    print(f"\n📈 SIMILITUD NORMALIZADA:")
    print("=" * 30)
    
    sim_tfidf_bm25 = tfidf_bm25_overlap / top_k
    sim_tfidf_faiss = tfidf_faiss_overlap / top_k
    sim_bm25_faiss = bm25_faiss_overlap / top_k
    sim_all_three = all_three_overlap / top_k
    
    print(f"TF-IDF vs BM25:    {sim_tfidf_bm25:.3f}")
    print(f"TF-IDF vs FAISS:   {sim_tfidf_faiss:.3f}")
    print(f"BM25 vs FAISS:     {sim_bm25_faiss:.3f}")
    print(f"Consenso (3 métodos): {sim_all_three:.3f}")
    
    # 4. Similitud promedio
    avg_similarity = (sim_tfidf_bm25 + sim_tfidf_faiss + sim_bm25_faiss) / 3
    print(f"\nSimilitud promedio: {avg_similarity:.3f}")
    
    # 5. Interpretación
    if avg_similarity >= 0.7:
        interpretation = "MUY ALTA similitud - Los métodos coinciden mucho"
    elif avg_similarity >= 0.5:
        interpretation = "ALTA similitud - Buen consenso entre métodos"
    elif avg_similarity >= 0.3:
        interpretation = "MEDIA similitud - Consenso moderado"
    else:
        interpretation = "BAJA similitud - Los métodos difieren significativamente"
    
    print(f"Interpretación: {interpretation}")

In [68]:
overlap_results = measure_ranking_overlap(query)

=== SIMILITUD ENTRE RANKINGS (Top-10) ===
Consulta: 'diphoton production cross sections'

📊 DOCUMENTOS COINCIDENTES:
TF-IDF ∩ BM25:     0/10 documentos (0.0%)
TF-IDF ∩ FAISS:    1/10 documentos (10.0%)
BM25 ∩ FAISS:      0/10 documentos (0.0%)
Común a los 3:     0/10 documentos (0.0%)

📈 SIMILITUD NORMALIZADA:
TF-IDF vs BM25:    0.000
TF-IDF vs FAISS:   0.100
BM25 vs FAISS:     0.000
Consenso (3 métodos): 0.000

Similitud promedio: 0.033
Interpretación: BAJA similitud - Los métodos difieren significativamente
